# PREPROCESAMIENTO DE LOS DATOS

In [1]:
from pyspark.sql import SparkSession
from pyspark import keyword_only
from pyspark.ml.param.shared import *
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.pipeline import Estimator, Model, Pipeline
import pyspark.sql.functions as p

In [2]:
spark = SparkSession.builder.appName("Pipeline Preprocesado").master("local").getOrCreate()

22/06/23 00:58:45 WARN Utils: Your hostname, lu-Aspire-A515-55 resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/06/23 00:58:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/23 00:58:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/23 00:58:46 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 5f:15:32:12:fe:9a:ab:0a
22/06/23 00:58:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


En este apartado vamos a realizar una serie de técnicas de preprocesamiento de datos que vamos a aplicar a nuestro conjunto de datos. Implementaremos cada paso de la Pipeline con la estructura de clases descrita de MLlib. Cada paso del preprocesamiento se desarrollará para que funcione de forma independiente y además después se incluirá en una Pipeline para poder aplicarlos también todos a la vez. Por último guardaremos dicha Pipeline para usarla cuando ajustemos modelos.

1) Detección de variables constantes

In [3]:
# Clase que inicializa el parámetro único
class HasColumnasConstantes(Params):
    def __init__(self):
        super(HasColumnasConstantes, self).__init__()
        self.columnasConstantes = Param(self, "columnasConstantes", "columnasConstantes", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasConstantes(self, value):
        return self.set(self.columnasConstantes, value)

    def getColumnasConstantes(self):
        return self.getOrDefault(self.columnasConstantes)

# Clase que define el estimador
class DeteccionConstantesEstimator(Estimator, DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self):
        super(DeteccionConstantesEstimator, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    @staticmethod
    def eliminacion_constantes(df):
        columnas = df.columns
        columnas_constantes = []
        s = len(df.columns)
        for i, columna in zip(range(s), columnas):
            valores_distintos = df.dropna(subset=[columna]).select(columna).distinct().count()
            if valores_distintos<=1:
                columnas_constantes.append(columna)
        return columnas_constantes
    
    def _fit(self, df):
        columnas_constantes = self. eliminacion_constantes(df)
        return DeteccionConstantesModel(columnasConstantes=columnas_constantes)

# Clase que define el transformer
class DeteccionConstantesModel(Model, HasColumnasConstantes,  DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self, columnasConstantes=[]):
        super(DeteccionConstantesModel, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, columnasConstantes=[]):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, df):
        columnas_constantes = self.getColumnasConstantes()
        try:
            df_final = df.drop(*columnas_constantes)
        except:
            df_final = list([])
        return df_final

2) Imputación de Missings

In [4]:
# Clases que inicializan los distintos parámetros del algoritmo
class HasColumnasMissingsImputarCategoricas(Params):
    def __init__(self):
        super(HasColumnasMissingsImputarCategoricas, self).__init__()
        self.columnasMissingsImputarCategoricas = Param(self, "columnasMissingsImputarCategoricas", 
                                                        "columnasMissingsImputarCategoricas", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasMissingsImputarCategoricas(self, value):
        return self.set(self.columnasMissingsImputarCategoricas, value)

    def getColumnasMissingsImputarCategoricas(self):
        return self.getOrDefault(self.columnasMissingsImputarCategoricas)

class HasColumnasMissingsImputarNumericas(Params):
    def __init__(self):
        super(HasColumnasMissingsImputarNumericas, self).__init__()
        self.columnasMissingsImputarNumericas = Param(self, "columnasMissingsImputarNumericas", 
                                                        "columnasMissingsImputarNumericas", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasMissingsImputarNumericas(self, value):
        return self.set(self.columnasMissingsImputarNumericas, value)

    def getColumnasMissingsImputarNumericas(self):
        return self.getOrDefault(self.columnasMissingsImputarNumericas)

class HasColumnasMissingsEliminar(Params):
    def __init__(self):
        super(HasColumnasMissingsEliminar, self).__init__()
        self.columnasMissingsEliminar = Param(self, "columnasMissingsEliminar", "columnasMissingsEliminar", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasMissingsEliminar(self, value):
        return self.set(self.columnasMissingsEliminar, value)

    def getColumnasMissingsEliminar(self):
        return self.getOrDefault(self.columnasMissingsEliminar)
    
class HasListaMissingsModas(Params):
    def __init__(self):
        super(HasListaMissingsModas, self).__init__()
        self.listaMissingsModas = Param(self, "listaMissingsModas", "listaMissingsModas", 
                                        typeConverter=TypeConverters.identity)
        
    def setListaMissingsModas(self, value):
        return self.set(self.listaMissingsModas, value)

    def getListaMissingsModas(self):
        return self.getOrDefault(self.listaMissingsModas)

class HasListaMissingsMedianas(Params):
    def __init__(self):
        super(HasListaMissingsMedianas, self).__init__()
        self.listaMissingsMedianas = Param(self, "listaMissingsMedianas", "listaMissingsMedianas", 
                                        typeConverter=TypeConverters.identity)
        
    def setListaMissingsMedianas(self, value):
        return self.set(self.listaMissingsMedianas, value)

    def getListaMissingsMedianas(self):
        return self.getOrDefault(self.listaMissingsMedianas)

class HasListaMissingsMedias(Params):
    def __init__(self):
        super(HasListaMissingsMedias, self).__init__()
        self.listaMissingsMedias = Param(self, "listaMissingsMedias", "listaMissingsMedias", 
                                        typeConverter=TypeConverters.identity)
        
    def setListaMissingsMedias(self, value):
        return self.set(self.listaMissingsMedias, value)

    def getListaMissingsMedias(self):
        return self.getOrDefault(self.listaMissingsMedias)
    
class HasPorcentajeMissings(Params):
    def __init__(self):
        super(HasPorcentajeMissings, self).__init__()
        self.porcentajeMissings = Param(self, "porcentajeMissings", "porcentajeMissings", 
                                        typeConverter=TypeConverters.identity)
        
    def setPorcentajeMissings(self, value):
        return self.set(self.porcentajeMissings, value)

    def getPorcentajeMissings(self):
        return self.getOrDefault(self.porcentajeMissings)

class HasMetodoImputacionNumericas(Params):
    def __init__(self):
        super(HasMetodoImputacionNumericas, self).__init__()
        self.metodoImputacionNumericas = Param(self, "metodoImputacionNumericas", "metodoImputacionNumericas", 
                                        typeConverter=TypeConverters.identity)
        
    def setMetodoImputacionNumericas(self, value):
        return self.set(self.metodoImputacionNumericas, value)

    def getMetodoImputacionNumericas(self):
        return self.getOrDefault(self.metodoImputacionNumericas)    

# Clase que define el estimador
class ImputacionMissingsEstimator(Estimator, HasPorcentajeMissings, HasMetodoImputacionNumericas, 
                                  DefaultParamsReadable, DefaultParamsWritable):
    
    @keyword_only
    def __init__(self, porcentajeMissings = 0.05, metodoImputacionNumericas = "media"):
        super(ImputacionMissingsEstimator, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self, porcentajeMissings = 0.05, metodoImputacionNumericas = "media"):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def deteccion_missings(self, df):
        porcentaje_missings = self.getPorcentajeMissings()
        columnas_missings_imputar_categoricas = []
        columnas_missings_imputar_numericas = []
        columnas_missings_eliminar = []
        df_size = df.count()
        if "target" in df.columns:
            columnas_sin_target = df.drop("target").columns
        else:
            columnas_sin_target = df.columns
        for c in columnas_sin_target:
            missing_porcentaje = df.select([(p.count(p.when(p.isnan(c) | p.col(c).isNull(), c))/df_size)
                                            .alias(c)]).collect()[0][0]
            if missing_porcentaje > porcentaje_missings:
                columnas_missings_eliminar.append(c)
            elif missing_porcentaje != 0:
                if df.select(c).dtypes[0][1].startswith('string'):
                    columnas_missings_imputar_categoricas.append(c)
                elif df.select(c).dtypes[0][1] in ["int", "float", "double", "long", "decimal"]:
                    columnas_missings_imputar_numericas.append(c)
        return columnas_missings_imputar_categoricas, columnas_missings_imputar_numericas, columnas_missings_eliminar
    
    @staticmethod
    def calcular_moda(df, columnas):
        lista_modas = []
        for columna in columnas:
            contador = df.groupby(columna).count().sort(p.col("count").desc())
            contador = contador.na.drop()
            moda = contador.first()[0]
            lista_modas.append(moda)
        return lista_modas
    
    @staticmethod
    def calcular_mediana(df, columnas):
        lista_medianas = df.approxQuantile(columnas, [0.5], 0.01)
        lista_medianas = [x for xs in lista_medianas for x in xs]
        return lista_medianas
       
    @staticmethod
    def calcular_media(df, columnas):
        lista_medias = list(df.select(p.avg(columna)).na.drop().collect()[0][0] for columna in columnas)
        return lista_medias
        
    def _fit(self, df):
        lista_modas=[]
        lista_medias=[]
        lista_medianas=[]
        columnas_missings_imputar_categoricas, columnas_missings_imputar_numericas, columnas_missings_eliminar = self.deteccion_missings(df)
        lista_modas = self.calcular_moda(df, columnas_missings_imputar_categoricas)
        metodo = self.getMetodoImputacionNumericas()
        if metodo == "Media":
            lista_medias = self.calcular_media(df, columnas_missings_imputar_numericas)
        if metodo == "Mediana":
            lista_medianas = self.calcular_mediana(df, columnas_missings_imputar_numericas)
        return ImputacionMissingsModel(columnasMissingsImputarCategoricas=columnas_missings_imputar_categoricas,
                                       columnasMissingsImputarNumericas=columnas_missings_imputar_numericas, 
                                       columnasMissingsEliminar=columnas_missings_eliminar,
                                       listaMissingsModas=lista_modas, listaMissingsMedianas=lista_medianas, 
                                       listaMissingsMedias=lista_medias, metodoImputacionNumericas=metodo)

# Clase que define el transformer
class ImputacionMissingsModel(Model, HasColumnasMissingsImputarCategoricas, HasColumnasMissingsImputarNumericas, 
                              HasColumnasMissingsEliminar, 
                              HasListaMissingsModas, HasMetodoImputacionNumericas, HasListaMissingsMedias,
                              HasListaMissingsMedianas, DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self, columnasMissingsImputarCategoricas=[], columnasMissingsImputarNumericas=[],
                 columnasMissingsEliminar=[], listaMissingsModas="", listaMissingsMedianas="", 
                 listaMissingsMedias="", metodoImputacionNumericas="media"):
        super(ImputacionMissingsModel, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, columnasMissingsImputarCategoricas=[], columnasMissingsImputarNumericas=[], 
                  columnasMissingsEliminar=[], listaMissingsModas="", listaMissingsMedianas="", 
                  listaMissingsMedias="", metodoImputacionNumericas="media"):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, df):
        columnas_missings_eliminar = self.getColumnasMissingsEliminar()
        columnas_missings_imputar_categoricas = self.getColumnasMissingsImputarCategoricas()
        columnas_missings_imputar_numericas = self.getColumnasMissingsImputarNumericas()
        for i in columnas_missings_eliminar:
            df = df.drop(i)
        for i in columnas_missings_imputar_categoricas:
            lista_modas = self.getListaMissingsModas()          
            diccionario_imputacion = dict(zip(columnas_missings_imputar_categoricas, lista_modas))
            df = df.fillna(diccionario_imputacion)
        metodo = self.getMetodoImputacionNumericas()
        for i in columnas_missings_imputar_numericas:
            if metodo == "Media":
                lista_medias = self.getListaMissingsMedias()
                diccionario_imputacion = dict(zip(columnas_missings_imputar_numericas, lista_medias))
                df = df.fillna(diccionario_imputacion)
            if metodo == "Mediana":
                lista_medianas = self.getListaMissingsMedianas()
                diccionario_imputacion = dict(zip(columnas_missings_imputar_numericas, lista_medianas))
                df = df.fillna(diccionario_imputacion)
        return df

3) Detección e imputación de Outliers

In [5]:
class HasColumnasOutliers(Params):
    def __init__(self):
        super(HasColumnasOutliers, self).__init__()
        self.columnasOutliers = Param(self, "columnasOutliers", "columnasOutliers", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasOutliers(self, value):
        return self.set(self.columnasOutliers, value)

    def getColumnasOutliers(self):
        return self.getOrDefault(self.columnasOutliers)
    
class HasMetodoOutliers(Params):
    def __init__(self):
        super(HasMetodoOutliers, self).__init__()
        self.metodoOutliers = Param(self, "metodoOutliers", "metodoOutliers", 
                                        typeConverter=TypeConverters.identity)
        
    def setMetodoOutliers(self, value):
        return self.set(self.metodoOutliers, value)

    def getMetodoOutliers(self):
        return self.getOrDefault(self.metodoOutliers)
    
class HasListaOutliersMedianas(Params):
    def __init__(self):
        super(HasListaOutliersMedianas, self).__init__()
        self.listaOutliersMedianas = Param(self, "listaOutliersMedianas", "listaOutliersMedianas", 
                                        typeConverter=TypeConverters.identity)
        
    def setListaOutliersMedianas(self, value):
        return self.set(self.listaOutliersMedianas, value)

    def getListaOutliersMedianas(self):
        return self.getOrDefault(self.listaOutliersMedianas)

class HasListaOutliersMedias(Params):
    def __init__(self):
        super(HasListaOutliersMedias, self).__init__()
        self.listaOutliersMedias = Param(self, "listaOutliersMedias", "listaOutliersMedias", 
                                        typeConverter=TypeConverters.identity)
        
    def setListaOutliersMedias(self, value):
        return self.set(self.listaOutliersMedias, value)

    def getListaOutliersMedias(self):
        return self.getOrDefault(self.listaOutliersMedias)

class HasBounds(Params):
    def __init__(self):
        super(HasBounds, self).__init__()
        self.bounds = Param(self, "bounds", "bounds", 
                                        typeConverter=TypeConverters.identity)
        
    def setBounds(self, value):
        return self.set(self.bounds, value)

    def getBounds(self):
        return self.getOrDefault(self.bounds)
    
# Clase que define el estimador
class OutliersEstimator(Estimator, HasMetodoOutliers, 
                                  DefaultParamsReadable, DefaultParamsWritable):
    
    @keyword_only
    def __init__(self, metodoOutliers = "media"):
        super(OutliersEstimator, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self, metodoOutliers = "media"):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    @staticmethod
    def calcular_mediana(df, columnas):
        lista_medianas = df.approxQuantile(columnas, [0.5], 0.01)
        lista_medianas = [x for xs in lista_medianas for x in xs]
        return lista_medianas
       
    @staticmethod
    def calcular_media(df, columnas):
        lista_medias = list(df.select(p.avg(columna)).na.drop().collect()[0][0] for columna in columnas)
        return lista_medias
    @staticmethod
    def calcular_limites(df, columnas):
        bounds = {
            c: dict(
                zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0.01))
            )
            for c in columnas
        }
        for c in bounds:
            iqr = bounds[c]['q3'] - bounds[c]['q1']
            bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
            bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)
        return bounds
    
    def _fit(self, df):
        lista_medias=[]
        lista_medianas=[]
        columnas_numericas=[]
        if "target" in df.columns:
            columnas_sin_target = df.drop("target").columns
        else:
            columnas_sin_target = df.columns
        for c in columnas_sin_target:
            if df.select(c).dtypes[0][1] in ["int", "float", "double", "long", "decimal"]:
                columnas_numericas.append(c)
        metodo = self.getMetodoOutliers()
        bounds = self.calcular_limites(df, columnas_numericas)
        if metodo == "Media":
            lista_medias = self.calcular_media(df, columnas_numericas)
        if metodo == "Mediana":
            lista_medianas = self.calcular_mediana(df, columnas_numericas)
        return OutliersModel(columnasOutliers=columnas_numericas,
                                       listaOutliersMedianas=lista_medianas, 
                                       listaOutliersMedias=lista_medias, metodoOutliers=metodo, bounds=bounds)

# Clase que define el transformer
class OutliersModel(Model, HasColumnasOutliers, HasListaOutliersMedianas, HasListaOutliersMedias, 
                    HasMetodoOutliers, HasBounds, DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self, columnasOutliers = "", listaOutliersMedianas="", listaOutliersMedias="", 
                 metodoOutliers="media", bounds=""):
        super(OutliersModel, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, columnasOutliers = "", listaOutliersMedianas="", listaOutliersMedias="", 
                  metodoOutliers="media", bounds=""):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, df):
        columnas_numericas = self.getColumnasOutliers()
        metodo = self.getMetodoOutliers()
        bounds = self.getBounds()
        if metodo == "Media":
            lista_medias = self.getListaOutliersMedias()
            for c in columnas_numericas:
                df=df.withColumn(c, p.when(
                            p.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
                            p.col(c)
                        ).otherwise(lista_medias[int(columnas_numericas.index(c))]))
        if metodo == "Mediana":
            lista_medianas = self.getListaOutliersMedianas()
            for c in columnas_numericas:
                df=df.withColumn(c, p.when(
                            p.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
                            p.col(c)
                        ).otherwise(lista_medianas[int(columnas_numericas.index(c))]))
   
        return df

4) Codificación de Variables Categóricas

In [6]:
class HasColumnasCod(Params):
    def __init__(self):
        super(HasColumnasCod, self).__init__()
        self.columnasCod = Param(self, "columnasCod", "columnasCod", 
                                        typeConverter=TypeConverters.identity)
        
    def setColumnasCod(self, value):
        return self.set(self.columnasCod, value)

    def getColumnasCod(self):
        return self.getOrDefault(self.columnasCod)
    
class HasMetodoCod(Params):
    def __init__(self):
        super(HasMetodoCod, self).__init__()
        self.metodoCod = Param(self, "metodoCod", "metodoCod", 
                                        typeConverter=TypeConverters.identity)
        
    def setMetodoCod(self, value):
        return self.set(self.metodoCod, value)

    def getMetodoCod(self):
        return self.getOrDefault(self.metodoCod)
    
class HasModeloCod(Params):
    def __init__(self):
        super(HasModeloCod, self).__init__()
        self.modeloCod = Param(self, "modeloCod", "modeloCod", 
                                        typeConverter=TypeConverters.identity)
        
    def setModeloCod(self, value):
        return self.set(self.modeloCod, value)

    def getModeloCod(self):
        return self.getOrDefault(self.modeloCod)
    
# Clase que define el estimador
class CodificacionCategoricasEstimator(Estimator, HasMetodoCod,
                                  DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self, metodoCod = "OneHotEncoder"):
        super(CodificacionCategoricasEstimator, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
    
    @keyword_only
    def setParams(self, metodoCod = "OneHotEncoder"):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _fit(self, df):
        columnas_categoricas=[]
        if "target" in df.columns:
            columnas_sin_target = df.drop("target").columns
        else:
            columnas_sin_target = df.columns
        for c in columnas_sin_target:
            if df.select(c).dtypes[0][1] in ["string"]:
                columnas_categoricas.append(c)
        metodo = self.getMetodoCod()
        if metodo == "OneHotEncoder":
            # Necesitamos aplicar previamente a OneHotEncoder una codificación adicional, StringIndexer.
            # Esto es necesario por la forma en que OneHotEncoder está implementado en MLlib.
            # Meteremos ambos estimadores en una pipeline.
            indexers = [StringIndexer(inputCol=columna, outputCol=columna+"-") for columna in columnas_categoricas]
            encoder = OneHotEncoder(
              inputCols=[indexer.getOutputCol() for indexer in indexers],
                outputCols=["{0}encoded".format(indexer.getOutputCol()) for indexer in indexers]
            )
            pipeline = Pipeline(stages=indexers + [encoder])
            encoder = pipeline.fit(df)
        if metodo == "StringIndexer":
            encoder = StringIndexer(inputCols=[columna for columna in columnas_categoricas], 
                                    outputCol=[columna+"-encoded" for columna in columnas_categoricas]) 

            encoder = encoder.fit(df)
        return CodificacionCategoricasModel(modeloCod=encoder, columnasCod=columnas_categoricas)

# Clase que define el transformer
class CodificacionCategoricasModel(Model, HasModeloCod, HasColumnasCod, 
                                   DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self, modeloCod = "", columnasCod=[]):
        super(CodificacionCategoricasModel, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, modeloCod = "", columnasCod=[]):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, df):
        modelo = self.getModeloCod()
        columnas_Cod = self.getColumnasCod()
        df = modelo.transform(df)
        df = df.drop(*columnas_Cod)
        return df

Una vez que hemos definido todos los metodos de preprocesamiento que vamos a aplicar a nuestro conjunto de datos, estamos listos para aplicarlos y obtener nuestro nuevo conjunto de datos trasnformado. Lo que haremos será definir las clases de los estimadores de los métodos seleccionando las opciones deseadas en cada caso y los meteremos todos en una pipeline.

In [7]:
deteccion_constantes = DeteccionConstantesEstimator()
missings = ImputacionMissingsEstimator(porcentajeMissings=0.5, metodoImputacionNumericas="Mediana")
outliers = OutliersEstimator(metodoOutliers="Mediana")
codificacion_categoricas = CodificacionCategoricasEstimator(metodoCod="OneHotEncoder")
pipeline = Pipeline(stages=[deteccion_constantes, missings, outliers, codificacion_categoricas])

De momento no aplicaremos el preprocesado a nuestro conjunto de datos pues en este Jupyter no vamos a realizar nada más. Lo guardaremos para utilizarlo en el Jupyter en el que ajustaremos los modelos de Machine Learning. Para guardar la pipeline utilizaremos el método save que nos permite guardarla con el formato de Spark. 

In [8]:
pipeline.write().overwrite().save("pipeline_preprocesado")